In [2]:
import sys
import os
import gc
# Get the absolute path to the main project folder
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add it to sys.path
sys.path.append(project_root)


import pickle
import numpy as np
import pandas as pd
from scipy import io
from scipy import io
import scipy
from scipy.stats import norm
import torch
from src.datasets.zipfian_dataset import create_dataloaders_zipfian
from src.datasets.uniform_dataset import create_dataloaders_uniform



from CCNL_readout_DBN import forwardrbm,forwardDBN,classifier, beta_extraction
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import accuracy_score
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Datasets statistics

In [ ]:
path_to_dbn="/home/student/Desktop/Groundeep/networks/zipfian"
name_of_dbn="idbn_trained_zipfian_1500_2000"
path_to_train_dataset='/home/student/Desktop/Groundeep/pairs_from_mat_train.pkl'
path_to_test_dataset= '/home/student/Desktop/Groundeep/pairs_from_mat_test.pkl'
test_file = '/home/student/Desktop/Groundeep/NumStim_1to32_100x100_TE.mat'
output_file = 'model_coefficients_results_5.xlsx'
# Prepare the results for saving into an Excel file
results = []
layer_size = [1000, 1500]

train_dataset = pickle.load(open(path_to_train_dataset, 'rb'))
XtrainComp, YtrainComp, idxs_train = train_dataset['data'].to(DEVICE), train_dataset['labels'].to(DEVICE), train_dataset['idxs'].to(DEVICE)
test_dataset = pickle.load(open(path_to_test_dataset, 'rb'))
XtestComp, YtestComp, idxs_test = test_dataset['data'].to(DEVICE), test_dataset['labels'].to(DEVICE), test_dataset['idxs'].to(DEVICE)
from scipy import io

input_path= f"{path_to_dbn}/{name_of_dbn}.pkl"
with open(input_path, 'rb') as f:
    dbn = pickle.load(f)

_XtrainComp1 = forwardDBN(dbn,XtrainComp[:, :, 0:10000]).clone()
_YtrainComp = YtrainComp.clone()
_XtrainComp2 = forwardDBN(dbn, XtrainComp[:, :, 10000:20000]).clone()
_XtrainComp = torch.cat((_XtrainComp1, _XtrainComp2), 2)
del _XtrainComp1, _XtrainComp2

_XtestComp1 = forwardDBN(dbn, XtestComp[:, :, 0:10000]).clone()
_YtestComp = YtestComp.clone()
_XtestComp2 = forwardDBN(dbn, XtestComp[:, :, 10000:20000]).clone()
_XtestComp = torch.cat((_XtestComp1, _XtestComp2), 2)
del _XtestComp1, _XtestComp2


accTR, predTR, acc, choice = classifier(_XtrainComp,  _XtestComp, _YtrainComp, _YtestComp)


test_contents = io.loadmat(test_file)

N_list_test = test_contents['N_list']
TSA_list_test = test_contents['TSA_list']
FA_list_test = test_contents['FA_list']

# Run the model for the current layer size
model_fit, weber, prob_choice_right, X = beta_extraction(
    choice, idxs_test, N_list_test, TSA_list_test, FA_list_test
)
# Collect results for the current layer size
results.append({
    'Layer Size': f"{layer_size[0]} {layer_size[1]}",
    'Intercept': model_fit,
    'Beta Number': weber[0],
    'Beta Size': weber[1],
    'Beta Spacing': weber[2],
    'Weber Fraction': prob_choice_right,
    "Accuracy": acc
})
print(results)

#Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save the DataFrame to an Excel file

results_df.to_excel(output_file, index=False)

print(f"Results saved to {output_file}")



del dbn, XtrainComp, YtrainComp, XtestComp, YtestComp
del _XtrainComp, _XtestComp, _YtrainComp, _YtestComp

# Run garbage collection
gc.collect()

# Free unused CUDA memory
torch.cuda.empty_cache()

(15200, 4000) (15200, 2) (15200, 4000) (15200, 2)
Pseudo-Inverse Perceptron:
Train Accuracy: 0.9203947368421053
Test Accuracy: 0.7752631578947369
--- Coefficients Debugging ---
Intercept: -6.9784225737319784e-09
Betas: [0.83326168 0.10021794 0.02047745]
Weber Fraction: 0.8486011036905079
[{'Layer Size': '1000 1500', 'Intercept': np.float64(-6.9784225737319784e-09), 'Beta Number': np.float64(0.8332616798533359), 'Beta Size': np.float64(0.10021793522364256), 'Beta Spacing': np.float64(0.020477445750146265), 'Weber Fraction': np.float64(0.8486011036905079), 'Accuracy': 0.7752631578947369}]
Results saved to model_coefficients_results_5.xlsx


: 